# 🎙️ Amir Voice Test - Ultimate TTS Playground

This notebook consolidates all Text-to-Speech engines for the **Su6i Yar** project.
Here you can test and compare different models side-by-side.

---

## 1. Setup & Imports
Run this cell first to load necessary libraries.

In [ ]:
import os
import sys
import time
import wave
import struct
import numpy as np
import json
from IPython.display import Audio, display

# Try importing Sherpa (Local Engine)
try:
    import sherpa_onnx
    SHERPA_AVAILABLE = True
    print("✅ sherpa-onnx found.")
except ImportError:
    SHERPA_AVAILABLE = False
    print("❌ sherpa-onnx not found. (Local TTS will be disabled)")

# Constants for Local Model
MODEL_PATH = "models/fa_IR-mana-medium-fixed.onnx"
ORIGINAL_CONFIG = "models/fa_IR-mana-medium.onnx.json"
TOKENS_PATH = "models/tokens.txt"
ESPEAK_DATA = "/opt/homebrew/share/espeak-ng-data"


## 2. 🧪 Engine: Local Piper (Sherpa-ONNX)
**Model:** Mana (Medium)  
**Speed:** Real-time on CPU (M1/M2)  
**Quality:** High

In [ ]:
# Initialize Sherpa Engine
if SHERPA_AVAILABLE and os.path.exists(MODEL_PATH):
    print("🚀 Initializing Sherpa-ONNX...")
    
    # Generate tokens if missing
    if not os.path.exists(TOKENS_PATH) and os.path.exists(ORIGINAL_CONFIG):
        with open(ORIGINAL_CONFIG, "r", encoding="utf-8") as f:
            data = json.load(f)
        id_map = data.get("phoneme_id_map", {})
        with open(TOKENS_PATH, "w", encoding="utf-8") as f:
            for symbol, ids in id_map.items():
                if ids:
                    f.write(f"{symbol} {ids[0]}\n")
        print("✅ tokens.txt generated.")

    try:
        config = sherpa_onnx.OfflineTtsConfig(
            model=sherpa_onnx.OfflineTtsModelConfig(
                vits=sherpa_onnx.OfflineTtsVitsModelConfig(
                    model=MODEL_PATH,
                    tokens=TOKENS_PATH,
                    data_dir=ESPEAK_DATA,
                    noise_scale=0.667,
                    length_scale=1.0,
                    noise_scale_w=0.8,
                ),
                provider="cpu",
                num_threads=1,
                debug=False
            )
        )
        tts_sherpa = sherpa_onnx.OfflineTts(config)
        print("✅ Sherpa Engine Ready!")
    except Exception as e:
        print(f"❌ Initialization Failed: {e}")
        tts_sherpa = None
else:
    print("⚠️ Sherpa engine skipped (missing files or lib).")
    tts_sherpa = None

def play_sherpa(text):
    if not tts_sherpa:
        print("❌ Engine not loaded.")
        return
        
    start = time.time()
    audio = tts_sherpa.generate(text, sid=0, speed=1.0)
    duration = time.time() - start
    print(f"⚡ Generated in {duration:.3f}s")
    
    samples = np.array(audio.samples)
    display(Audio(samples, rate=audio.sample_rate, autoplay=True))

In [ ]:
# Test Sherpa
play_sherpa("سلام امیرجان. این تست صدای مانا روی مک‌بوک شماست. سرعت را ببینید!")

## 3. ☁️ Engine: Datacula (Amir)
**Model:** Amir  
**Type:** Online API  
**Latency:** Depends on network

In [ ]:
import requests

def play_datacula(text):
    url = "https://tts.datacula.com/api/tts"
    params = {"text": text, "model_name": "امیر"}
    
    print(f"☁️ Requesting Datacula API...")
    start = time.time()
    try:
        resp = requests.get(url, params=params, timeout=10)
        if resp.status_code == 200:
            print(f"✅ Received in {time.time()-start:.2f}s")
            # Save to temporary file for playing
            with open("datacula_test.wav", "wb") as f:
                f.write(resp.content)
            display(Audio("datacula_test.wav", autoplay=False))
        else:
            print(f"❌ Failed: {resp.status_code}")
    except Exception as e:
        print(f"❌ Error: {e}")

# play_datacula("سلام. این صدای امیر از داتاکولا است.")

## 4. 🧠 Engine: EdgeTTS (Farid)
**Model:** Microsoft Farid (Online)  
**Quality:** Standard, slightly robotic

In [ ]:
# EdgeTTS requires asyncio in notebook which can be tricky, 
# or we call it via subprocess cli for simplicity in notebook.

def play_edge(text):
    output_file = "edge_test.mp3"
    cmd = f'edge-tts --text "{text}" --voice fa-IR-FaridNeural --write-media {output_file}'
    
    print("🧠 Running EdgeTTS...")
    res = os.system(cmd)
    if res == 0:
        display(Audio(output_file, autoplay=False))
    else:
        print("❌ EdgeTTS failed.")

# play_edge("سلام. من فرید هستم از مایکروسافت.")

---